In [ ]:
# packages needed
import pandas as pd
import numpy as np

import ktrain
from ktrain import text
import nltk
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score

from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata

# data preparation

In [ ]:
# load data
%store -r data_train_taskc data_test_taskc

In [ ]:
data_train_taskc['label_vector'].value_counts()

In [ ]:
# visualize the label distribution for taskc
import matplotlib.pyplot as plt
# Data
categories = [
    
    "2.1 descriptive attacks",
    "2.2 aggressive and emotive attacks",
    "3.1 casual use of gendered slurs, profanities, and insults",
    "3.2 immutable gender differences and gender stereotypes",
    "2.3 dehumanising attacks & overt sexual objectification",
    "4.2 supporting systemic discrimination against women as a group",
    "1.2 incitement and encouragement of harm",
    "4.1 supporting mistreatment of individual women",
    "3.4 condescending explanations or unwelcome advice",
    "3.3 backhanded gendered compliments",
    "1.1 threats of harm"
]

counts = [
     3281, 2581, 2472, 1472, 1042, 1026, 924, 437, 349, 285, 258
]

# Create a horizontal bar plot
plt.figure(figsize=(3, 2.4))
plt.barh(categories, counts, color='skyblue')
plt.xlabel('Count')
plt.title('Category Counts')
plt.gca().invert_yaxis()  # Invert y-axis to show the highest count at the top
plt.show()

In [ ]:
# split data to training and test set
x = data_train_taskc.text.tolist()
y = data_train_taskc.label_vector.tolist()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 42)

In [ ]:
#encoding data for task C
encoding = {
    '1.1 threats of harm': 0, 
    '1.2 incitement and encouragement of harm': 1,
    '2.1 descriptive attacks': 2, 
    '2.2 aggressive and emotive attacks': 3,
    '2.3 dehumanising attacks & overt sexual objectification' : 4, 
    '3.1 casual use of gendered slurs, profanities, and insults': 5,
    '3.2 immutable gender differences and gender stereotypes':6,
    '3.3 backhanded gendered compliments': 7, 
    '3.4 condescending explanations or unwelcome advice':8,
    '4.1 supporting mistreatment of individual women':9,
    '4.2 supporting systemic discrimination against women as a group': 10
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

# focal loss

In [ ]:
import tensorflow as tf
from tensorflow.keras import activations
def focal_loss(gamma=2., alpha=4., from_logits=False):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax if from_logits is False.
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        if from_logits:
            y_pred = activations.softmax(y_pred)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

# 11 sub-categories classification

In [ ]:
BATCH_SIZE = 6

## Electra

In [ ]:
model_1 = 'google/electra-base-discriminator'

In [ ]:
t_1 = text.Transformer(model_1, maxlen=500, class_names=[0,1,2,3,4,5,6,7,8,9,10])
trn_1 = t_1.preprocess_train(x_train, y_train)
val_1 = t_1.preprocess_test(x_test, y_test)

model_electra = t_1.get_classifier()

In [ ]:
model_electra.compile(loss=focal_loss(alpha=1, from_logits=True),optimizer='adam',metrics=['accuracy'])

In [ ]:
learner_electra = ktrain.get_learner(model_electra, train_data=trn_1, val_data=val_1, batch_size =BATCH_SIZE)

In [ ]:
learner_electra.fit_onecycle(2e-5, 4) 

## Roberta

In [ ]:
model_2 = 'roberta-base' 

In [ ]:
t_roberta = text.Transformer(model_2, maxlen=500, class_names=[0, 1,2,3,4,5,6,7,8,9,10])
trn_roberta = t_roberta.preprocess_train(x_train, y_train)
val_roberta = t_roberta.preprocess_test(x_test, y_test)

model_roberta = t_roberta.get_classifier()

In [ ]:
model_roberta.compile(loss=focal_loss(alpha=1, from_logits=True),optimizer='adam',metrics=['accuracy'])

In [ ]:
learner_roberta = ktrain.get_learner(model_roberta, train_data=trn_roberta, val_data=val_roberta, batch_size =BATCH_SIZE)

In [ ]:
learner_roberta.fit_onecycle(2e-5, 4) #use hardware acceleartor while running this cell

## Deberta

In [ ]:
model_3 = 'microsoft/deberta-base' 

In [ ]:
t_3 = text.Transformer(model_3, maxlen=500, class_names=[0, 1,2,3,4,5,6,7,8,9,10])
trn_3 = t_3.preprocess_train(x_train, y_train)
val_3 = t_3.preprocess_test(x_test, y_test)

model_deberta = t_3.get_classifier()

In [ ]:
model_deberta.compile(loss=focal_loss(alpha=1, from_logits=True),optimizer='adam',metrics=['accuracy'])

In [ ]:
learner_deberta = ktrain.get_learner(model_deberta, train_data=trn_roberta, val_data=val_roberta, batch_size =BATCH_SIZE)

In [ ]:
learner_deberta.fit_onecycle(2e-5, 4) #use hardware acceleartor while running this cell